In [19]:
!pip install streamlit

In [1]:
%%writefile app.py
import streamlit as st
import sqlite3
import os
import google.generativeai as genai
import os
os.environ['API_KEY']=#gemini api
# Configure Google Generative AI
genai.configure(api_key=os.getenv('API_KEY'))
model_1 = genai.GenerativeModel('gemini-1.5-flash')
model = model_1.start_chat(history=[])
# Create a database connection
conn = sqlite3.connect("chat.db", check_same_thread=False)
cursor = conn.cursor()

# Create tables to store users and chat messages if they do not exist
cursor.execute("""
CREATE TABLE IF NOT EXISTS users (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    username TEXT UNIQUE,
    password TEXT
)
""")
cursor.execute("""
CREATE TABLE IF NOT EXISTS chats (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER,
    message TEXT,
    sender TEXT,
    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (user_id) REFERENCES users (id)
)
""")
conn.commit()

# Helper function to create a new user
def create_user(username, password):
    try:
        cursor.execute("INSERT INTO users (username, password) VALUES (?, ?)", (username, password))
        conn.commit()
    except sqlite3.IntegrityError:
        st.error("Username already exists.")

# Helper function to authenticate a user
def authenticate_user(username, password):
    cursor.execute("SELECT id FROM users WHERE username = ? AND password = ?", (username, password))
    return cursor.fetchone()

# Helper function to add a message to the chat history
def add_message(user_id, message, sender):
    cursor.execute("INSERT INTO chats (user_id, message, sender) VALUES (?, ?, ?)", (user_id, message, sender))
    conn.commit()

# Helper function to fetch chat history
def fetch_chat_history(user_id):
    cursor.execute("SELECT message, sender FROM chats WHERE user_id = ? ORDER BY timestamp", (user_id,))
    return cursor.fetchall()

# Streamlit app
st.title("Chat App with LLM Integration")

# State management for user session
if 'logged_in' not in st.session_state:
    st.session_state.logged_in = False
    st.session_state.user_id = None
    st.session_state.page = 'login'

# Helper function to handle login
def handle_login():
    username = st.text_input("Username", key="login_username")
    password = st.text_input("Password", type="password", key="login_password")
    if st.button("Login"):
        user = authenticate_user(username, password)
        if user:
            st.session_state.logged_in = True
            st.session_state.user_id = user[0]
            st.session_state.username = username
            st.session_state.page = 'chat'
            st.experimental_rerun()
        else:
            st.error("Invalid username or password.")

# Helper function to handle signup
def handle_signup():
    new_username = st.text_input("Username", key="signup_username")
    new_password = st.text_input("Password", type="password", key="signup_password")
    if st.button("Signup"):
        if new_username and new_password:
            create_user(new_username, new_password)
            st.success("User created successfully! Please log in.")
        else:
            st.error("Please enter both username and password.")

# Helper function to show chat interface
def show_chat():
    st.subheader(f"Chat as {st.session_state.username}")
    user_input = st.text_area("Your message:", key="user_input")

    if st.button("Send"):
        if user_input:
            add_message(st.session_state.user_id, user_input, sender="user")
            # Generate a response from the LLM
            response = model.send_message(user_input).text
            add_message(st.session_state.user_id, response, sender="assistant")
            st.experimental_rerun()  # To update the chat history

    # Display chat history for the logged-in user
    chat_history = fetch_chat_history(st.session_state.user_id)
    for message, sender in chat_history:
        sender_name = "You" if sender == "user" else "Assistant"
        st.write(f"{sender_name}: {message}")

    # Logout option
    if st.button("Logout"):
        st.session_state.logged_in = False
        st.session_state.user_id = None
        st.session_state.page = 'login'
        st.experimental_rerun()

# Display the appropriate interface based on the state
if not st.session_state.logged_in:
    option = st.selectbox("Login or Signup", ["Login", "Signup"])
    if option == "Login":
        handle_login()
    elif option == "Signup":
        handle_signup()
else:
    show_chat()

# Close the database connection
conn.close()


Writing app.py


In [21]:
! wget -q -O - ipv4.icanhazip.com

35.226.246.149


In [23]:
! streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.226.246.149:8501

npx: installed 22 in 3.086s
your url is: https://fine-clocks-matter.loca.lt
2024-07-04 13:16:38.752 Please replace `st.experimental_rerun` with `st.rerun`.

`st.experimental_rerun` will be removed after 2024-04-01.
2024-07-04 13:16:57.382 Please replace `st.experimental_rerun` with `st.rerun`.

`st.experimental_rerun` will be removed after 2024-04-01.
2024-07-04 13:17:10.448 Please replace `st.experimental_rerun` with `st.rerun`.

`st.experimental_rerun` will be removed after 2024-04-01.
2024-07-04 13:17:18.865 Please replace `st.experimental_rerun` with `st.rerun`.

`st.experimental_rerun` will be removed after 2024-04-01.
2024-07-04 13:21:26.984 Please replace `st.experimental_rerun` with `st.rerun`.

`st.experimental_rerun` will be removed after 2024-04-01.
2024-07-04 13:21:51.123 Please replace `st.experimen

In [ ]:

# # Initialize the pipeline for text generation
# def initialize_pipeline():
#     model_name = "microsoft/Phi-3-mini-128k-instruct"
#     tokenizer = AutoTokenizer.from_pretrained(model_name)
#     model = AutoModelForCausalLM.from_pretrained(
#         model_name,
#         torch_dtype=torch.float32,
#         trust_remote_code=True,
#     )
#     pipe = pipeline(
#         "text-generation",
#         model=model,
#         tokenizer=tokenizer,
#         device=0 if torch.cuda.is_available() else -1  # Use GPU if available, else CPU
#     )
#     return pipe

In [ ]:

# import streamlit as st
# import sqlite3

# # Create a database connection
# conn = sqlite3.connect("chat.db")
# cursor = conn.cursor()

# # Create a table to store the chat messages
# cursor.execute("""
# CREATE TABLE IF NOT EXISTS chats (
#     id INTEGER PRIMARY KEY AUTOINCREMENT,
#     user TEXT,
#     message TEXT,
#     timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
# )
# """)
# conn.commit()

# # Define the Streamlit app
# st.title("Simple Chat App")

# # Get the user's input
# user_input = st.text_area("Your message:", "")

# # If the user has submitted a message, add it to the database
# if st.button("Send"):
#     if user_input:
#         cursor.execute("INSERT INTO chats (user, message) VALUES (?, ?)", ("user", user_input))
#         conn.commit()
#         st.write("Message sent!")

# # Display the chat history
# cursor.execute("SELECT user, message FROM chats ORDER BY timestamp")
# for row in cursor.fetchall():
#     st.write(f"{row[0]}: {row[1]}")

# # Close the database connection
# conn.close()


In [ ]:
# import streamlit as st
# import sqlite3
# from datetime import datetime
# from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
# import torch

# tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")
# model = AutoModelForCausalLM.from_pretrained(
#     "microsoft/Phi-3-mini-128k-instruct",

#     torch_dtype="auto",
#     trust_remote_code=True,
# )
# pipe = pipeline(
#         "text-generation",
#         model=model,
#         tokenizer=tokenizer,
#         device=0 )


In [ ]:
# import streamlit as st
# import sqlite3
# from datetime import datetime
# from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
# import torch


# # Initialize the pipeline for text generation

# model_name = "microsoft/Phi-3-mini-128k-instruct"
# tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")

# model = AutoModelForCausalLM.from_pretrained(
#         model_name,
#         torch_dtype=torch.float32,
#         trust_remote_code=True
#     )
# pipe = pipeline(
#         "text-generation",
#         model=model,
#         tokenizer=tokenizer,
#         device=0 )

In [ ]:
# import torch
# from transformers import pipeline

# generate_text = pipeline(model="databricks/dolly-v2-3b", torch_dtype=torch.bfloat16, trust_remote_code=True, device_map="auto")


In [ ]:

# from transformers import T5Tokenizer, T5ForConditionalGeneration

# tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small")
# model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small")

# input_text = "can you tell difference between astrology and astronomy"
# input_ids = tokenizer(input_text, return_tensors="pt").input_ids

# outputs = model.generate(input_ids)
# print(tokenizer.decode(outputs[0]))


In [ ]:
# !pip uninstall torch transformers
# !pip install torch transformers


In [ ]:
# from transformers import pipeline

# generator = pipeline("text-generation", model="gpt2")
# print(generator("Hello, I'm a transformer model")[0]['generated_text'])


In [ ]:
# import torch
# from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline


In [ ]:

# !pip install accelerate

# model = AutoModelForCausalLM.from_pretrained(
#     "microsoft/Phi-3-mini-128k-instruct",

#     torch_dtype="auto",
#     trust_remote_code=True,
# )

In [ ]:




# tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")

# messages = [
#       {"role": "user", "content": "What about solving an 2x + 3 = 7 equation?"},
# ]



# generation_args = {
#     "max_new_tokens": 500,
#     "return_full_text": False,
#     "temperature": 0.0,
#     "do_sample": False,
# }

# output = pipe(messages, **generation_args)
# print(output[0]['generated_text'])


In [ ]:
# import torch
# from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# # Initialize model and tokenizer
# model_name = "microsoft/Phi-3-mini-128k-instruct"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name)

# # Initialize the pipeline with the GPU device (0 for the first GPU)
# pipe = pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     device=0  # Use 0 to specify the first GPU
# )

# # Initial message and context
# messages = [
#     {"role": "user", "content": "What about solving an 2x + 3 = 7 equation?"}
# ]

# # Define generation arguments
# generation_args = {
#     "max_new_tokens": 500,
#     "return_full_text": False,
#     "temperature": 0.0,
#     "do_sample": False,
# }

# # Function to maintain conversation context
# def generate_response(pipe, messages, generation_args):
#     # Format the input for the model
#     formatted_input = [{"role": m["role"], "content": m["content"]} for m in messages]

#     # Generate text
#     output = pipe(formatted_input, **generation_args)

#     # Extract and return the generated text
#     return output[0]['generated_text']

# # Generate the first response
# response = generate_response(pipe, messages, generation_args)
# print("Model:", response)

# # Append the model's response to the conversation history
# messages.append({"role": "assistant", "content": response})

# # Continue the conversation
# # Example: Add more user input
# new_user_input = "Can you explain the solution?"
# messages.append({"role": "user", "content": new_user_input})

# # Generate the next response
# response = generate_response(pipe, messages, generation_args)
# print("Model:", response)

# # Update the conversation history
# messages.append({"role": "assistant", "content": response})


In [ ]:
#  model = AutoModelForCausalLM.from_pretrained(
#     "microsoft/Phi-3-mini-128k-instruct",

#     torch_dtype="auto",
#     trust_remote_code=True,  )


In [ ]:
# model_name = "microsoft/Phi-3-mini-128k-instruct"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# pipe = pipeline(
#         "text-generation",
#         model=model,
#         tokenizer=tokenizer,
#         device=0  # Use GPU (0 for the first GPU) or -1 for CPU
#     )

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# import pickle
# # Save the pipe variable to a file in Google Drive
# with open('/content/drive/MyDrive/LLM_chat/pipe.pkl', 'wb') as f:
#     pickle.dump(pipe, f)



In [ ]:
# # Load the pipe variable from Google Drive
# with open('/content/drive/MyDrive/LLM_chat/pipe.pkl', 'rb') as f:
#     pipe = pickle.load(f)


In [ ]:
# import torch
# from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# # Function to initialize the pipeline
# def initialize_pipeline(device=0):
#   # Load the pipe variable from Google Drive

#     model_name = "microsoft/Phi-3-mini-128k-instruct"
#     tokenizer = AutoTokenizer.from_pretrained(model_name)
#     model = AutoModelForCausalLM.from_pretrained(
#     "microsoft/Phi-3-mini-128k-instruct",

#     torch_dtype="auto",
#     trust_remote_code=True,  )


#     pipe = pipeline(
#         "text-generation",
#         model=model,
#         tokenizer=tokenizer,
#         device=0  # Use GPU (0 for the first GPU) or -1 for CPU
#     )
#     return pipe

# # Function to add user input to the conversation history
# def add_user_input(messages, user_input):
#     messages.append({"role": "user", "content": user_input})
#     return messages

# # Function to generate a response from the model
# def generate_response(pipe, messages, generation_args):
#     formatted_input = [{"role": m["role"], "content": m["content"]} for m in messages]
#     output = pipe(formatted_input, **generation_args)
#     generated_text = output[0]['generated_text']
#     messages.append({"role": "assistant", "content": generated_text})
#     return generated_text

# # Function to print the conversation
# def print_conversation(messages):
#     for message in messages:
#         role = "User" if message["role"] == "user" else "Assistant"
#         print(f"{role}: {message['content']}\n")

# # Main execution
# if __name__ == "__main__":

#     # Initialize the pipeline
#     pipe = initialize_pipeline()

#     # Initial conversation history
#     messages = []

#     # Define generation arguments
#     generation_args = {
#         "max_new_tokens": 500,
#         "return_full_text": False,
#         "temperature": 0.0,
#         "do_sample": False,
#     }

#     # First user input
#     user_input = "What about solving a 2x + 3 = 7 equation?"
#     messages = add_user_input(messages, user_input)

#     # Generate and print response
#     response = generate_response(pipe, messages, generation_args)
#     print_conversation(messages)

#     # Additional user input
#     user_input = "Can you explain the solution?"
#     messages = add_user_input(messages, user_input)

#     # Generate and print response
#     response = generate_response(pipe, messages, generation_args)
#     print_conversation(messages)


In [ ]:
# !pip install --upgrade openai

In [ ]:
# # import the OpenAI Python library for calling the OpenAI API
# from openai import OpenAI

# import os

# client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "openapikey"))

In [ ]:
#  vertexai.init(project="", location="asia-south1")
#   model = GenerativeModel(
#     "gemini-1.5-flash-001",
#   )
#   responses = model.generate_content(
#       [],
#       generation_config=generation_config,
#       safety_settings=safety_settings,
#       stream=True,
#   )

#   for response in responses:
#     print(response.text, end="")




In [ ]:
# import vertexai
# from vertexai.preview.language_models import TextGenerationModel

# # Authenticate to GCP (if you haven't already)
# !gcloud auth application-default login


In [ ]:
# generation_config = {
#     "max_output_tokens": 250,
#     "temperature": 1,
#     "top_p": 0.95,
# }

In [ ]:

# import google.generativeai as genai
# import os

# genai.configure(api_key=os.environ['API_KEY'])
# model = genai.GenerativeModel('gemini-1.5-flash')
# chat = model.start_chat(history=[])

# response = chat.send_message("Which is one of the best place to visit in India during summer?")
# print(response.text)
# response = chat.send_message("Tell me more about that place in 50 words")
# print(response.text)


It's tough to pick just one "best" place, as India offers so much variety! It really depends on what you're looking for in a summer escape. 

**Here are some top contenders, each with their own unique appeal:**

**For cool mountain air and breathtaking views:**

* **Manali, Himachal Pradesh:** Known for its stunning landscapes, apple orchards, and adventure activities like trekking and paragliding.
* **Darjeeling, West Bengal:** Famous for its tea plantations, toy train, and panoramic views of the Himalayas.
* **Ladakh, Jammu & Kashmir:** A high-altitude desert with rugged mountains, ancient monasteries, and opportunities for trekking and motorcycle touring.
* **Shimla, Himachal Pradesh:** A historic hill station with colonial architecture, bustling markets, and scenic walks.

**For coastal breezes and beaches:**

* **Goa:** A popular beach destination with sandy shores, vibrant nightlife, and Portuguese architecture.
* **Kerala:** Known as "God's Own Country," Kerala offers serene bac